In [15]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import os
import pandas as pd
import torch
from sklearn.naive_bayes import GaussianNB, ComplementNB
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score

In [64]:
codes = np.array([1,2,2,2,2,1,1,1,1,3,1,1,1,3,1,1,1,3,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,
        1,1,1,3,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
known = [True] * 85
known[63] = False
known[17:20] = [False] * 3
known[48:58] = [False] * 10

In [65]:
def get_data():
    texts = []
    textprepath = "Federalist Papers/fed paper "
    texts = []
    for i in range(85):
        f = open(f"{textprepath}{i + 1}.txt", "r")
        text = f.read()
        parts = text.split("\n", 1) #split out the first line
        f.close()
        texts.append({'text':parts[1]})
    df = pd.DataFrame(texts)
    return df

In [66]:
data = get_data()
train = known and [i % 8 != 0 for i in range(len(known))]
test = known and [i % 8 == 0 for i in range(len(known))]
train_df = data[train]
test_df = data[test]
codestrain = codes[train]
codestest = codes[test]

In [46]:
#authors are tolstoy, beliaev, teffi, danilevskii, averchenko, chiornyi
#authors = np.repeat([1, 2, 2, 3, 4, 5, 4, 5, 6], 3)
authors = np.repeat([1, 3, 2, 2, 3, 3, 2, 2, 3, 5, 4, 5, 3, 5, 2, 5, 6, 3, 5], 3)
#huggingface, google, deepl
translators = np.repeat([1, 2, 3],19)
fullds =[] 
directory = "russian short stories"
directories = ['init translations', 'google translation', 'deepl translations']
#directory = "test translation"
#files = ["Сказка о лысом пророке Елисее chiornyi.txt"]
for directory in directories:
    #print(directory)
    for file in os.listdir(directory):
#for file in files:
        filename = os.fsdecode(file)
        if filename.endswith(".txt"):
            #print(filename)
            #f = open(f"{directory}/{filename}", encoding='utf-8') #'r',
            f = open(f"{directory}/{filename}", 'r',encoding='utf-8')
            text = f.read()
            f.close()
            fullds.append({'text':text})
            #fullds.append(text)
translated_df = pd.DataFrame(fullds)

In [47]:
#intrain = [i % 5 != 0 for i in range(translated_df.size)]
intrain = [i % 9 > 2 for i in range(translated_df.size)]
#intrain = [True, True, True, True, True, True, False, False, False,
#          True, True, True, False, False, False, True, True, True, 
#          False, False, False, True, True, True, True, True, True,]
#notintrain = [i % 9 <= 2 for i in range(translated_df.size)]
notintrain = [not element for element in intrain]
train_df = translated_df[intrain]
test_df = translated_df[notintrain]
codestrain = authors[intrain]
#codestrain = translators[intrain]
codestest = authors[notintrain]
#codestest = translators[notintrain]

In [67]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = iter(train_df['text'])
test_iter = iter(test_df['text'])
def yield_tokens(train_iter):
    for text in train_iter:
        yield tokenizer(text)

In [68]:
next(train_iter)[:10]

'\nConcernin'

In [69]:
# Only inlcude words that occur at least 10 times in the training data.
# Also let "<unk>" represent unknown words, i.e., words not in the vocabulary.
#vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"], min_freq=10)
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"], min_freq=2)

In [70]:
from itertools import combinations
from torchtext.vocab import GloVe

# It is best to save GloVe data in a cache to reuse across projects.
VECTOR_CACHE_DIR = '.vector_cache'

glove = GloVe(name='6B', cache = VECTOR_CACHE_DIR)
words = ["hello", "hi", "king", "president"]
vecs = glove.get_vecs_by_tokens(words)

In [33]:
def getfscore(clf, labels, inputs):
    pred = clf.predict(inputs)
    f = f1_score(labels, pred, average='micro')
    print(pred)
    print(labels)
    print(f)

In [12]:
def collate_into_cnbow(batch):
    k = len(batch)
    m = vecs.shape[1]
    tt = torch.zeros([k,m])#, dtype = torch.float)
    #for each word get pretrained embeddings
    for i in range(k):
        tt[i] = sum(glove.get_vecs_by_tokens(batch[i].split()))
    #print(tt[0,:5]/max(torch.abs(tt[1])))
    return torch.nn.functional.normalize(tt, dim = 1) + 1

In [9]:
def collate_into_cbow(batch):
    k = len(batch)
    m = vecs.shape[1]
    tt = torch.zeros([k,m])#, dtype = torch.float)
    #for each word get pretrained embeddings
    for i in range(k):
        tt[i] = sum(glove.get_vecs_by_tokens(batch[i].split()))
    return tt

In [10]:
def collate_into_bow(batch):
    k = len(batch)
    m = vocab.__len__()
    tt = torch.zeros([k,m])#, dtype = torch.float)
    for i in range(k):
        words = batch[i].split()
        for word in words:
            if (vocab.__contains__(word)):
                tt[i][vocab.__getitem__(word)] += 1/len(words)
    return tt

In [44]:

print(len(test_df['text']))
print(collate_into_cbow(list(test_df['text'])))

21
tensor([[ -97.5652,   86.8800,  -21.2476,  ..., -101.2449, -103.1667,
           46.4730],
        [-129.7837,  104.1886,  -45.2938,  ..., -174.8952,  -90.4316,
          104.2629],
        [-233.7854,  208.7918, -126.9739,  ..., -269.0349, -219.9509,
          117.1081],
        ...,
        [-126.4933,  107.4950,  -49.5160,  ..., -153.6434, -103.4746,
           73.0541],
        [-126.1103,  102.3025,  -53.6391,  ..., -200.8259, -105.9677,
          102.4004],
        [ -88.5561,  103.8365,  -20.1227,  ..., -151.1780,  -84.3824,
           59.2007]])


In [73]:
#translators/federalist papers
#xtrain = collate_into_cbow(list(train_df['text']))
xtrain = collate_into_cbow(list(train_df['text']))
clf = LogisticRegression(random_state=0, max_iter = 1000).fit(xtrain, codestrain)
xtest = collate_into_cbow(list(test_df['text']))
#clf.score(xtest, codestest)
getfscore(clf, codestest, xtest)

[1 1 1 1 1 1 3 3 1 1 1]
[1 1 1 1 1 3 3 3 1 1 1]
0.9090909090909091


In [47]:
#intrain = [i % 5 != 0 for i in range(translated_df.size)]
intrain = [i % 9 > 2 for i in range(translated_df.size)]
#intrain = [True, True, True, True, True, True, False, False, False,
#          True, True, True, False, False, False, True, True, True, 
#          False, False, False, True, True, True, True, True, True,]
#notintrain = [i % 9 <= 2 for i in range(translated_df.size)]
notintrain = [not element for element in intrain]
train_df = translated_df[intrain]
test_df = translated_df[notintrain]
codestrain = authors[intrain]
#codestrain = translators[intrain]
codestest = authors[notintrain]
#codestest = translators[notintrain]

In [48]:
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"], min_freq=10)

In [49]:
#xtrain = collate_into_cbow(list(train_df['text']))
xtrain = collate_into_bow(list(train_df['text']))
clf = LogisticRegression(random_state=0, max_iter = 10000).fit(xtrain, codestrain)

In [50]:
#xtest = collate_into_cbow(list(test_df['text']))
xtest = collate_into_bow(list(test_df['text']))
clf.score(xtest, codestest)

0.14285714285714285

In [75]:
collate_into_cnbow(list(train_df['text']))#.shape

tensor([[0.9718, 1.0331, 0.9713,  ..., 0.9487, 0.9690, 1.0167],
        [0.9623, 1.0266, 0.9746,  ..., 0.9461, 0.9750, 1.0369],
        [0.9642, 1.0305, 0.9860,  ..., 0.9489, 0.9650, 1.0253],
        ...,
        [0.9702, 1.0277, 0.9936,  ..., 0.9510, 0.9662, 1.0180],
        [0.9674, 1.0326, 0.9837,  ..., 0.9397, 0.9687, 1.0076],
        [0.9584, 1.0264, 0.9794,  ..., 0.9409, 0.9689, 1.0117]])

In [75]:
#for id translators
#codestrain = translators[intrain]
#codestest = translators[notintrain]
#clf = GaussianNB()
clf = ComplementNB(force_alpha=True)
xtrain = collate_into_bow(list(train_df['text']))
xtest = collate_into_bow(list(test_df['text']))
clf.fit(xtrain, codestrain)
clf.score(xtest, codestest)
getfscore(clf, codestest, xtest)

[1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 3 3 3 1 1 1]
0.7272727272727273


In [61]:
#for id translators
codestrain = translators[intrain]
codestest = translators[notintrain]
#clf = GaussianNB()
clf = ComplementNB(force_alpha=True)
xtrain = collate_into_bow(list(train_df['text']))
xtest = collate_into_bow(list(test_df['text']))
clf.fit(xtrain, codestrain)
clf.score(xtest, codestest)
getfscore(clf, codestest, xtest)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3]
0.3333333333333333


In [ ]:
codestrain = translators[intrain]
codestest = translators[notintrain]
#clf = GaussianNB()
clf = ComplementNB(force_alpha=True)
xtrain = collate_into_bow(list(train_df['text']))
xtest = collate_into_bow(list(test_df['text']))
clf.fit(xtrain, codestrain)
clf.score(xtest, codestest)

In [63]:
#for id original author
codestrain = authors[intrain]
codestest = authors[notintrain]
xtrain = collate_into_cnbow(list(train_df['text']))
xtest = collate_into_cnbow(list(test_df['text']))
clf = GaussianNB()
#clf = ComplementNB(force_alpha=True)
clf.fit(xtrain, codestrain)
clf.score(xtest, codestest)
getfscore(clf, codestest, xtest)

[3 2 2 3 3 2 3 3 2 2 5 6 2 3 3 3 2 5 5 2 3]
[1 1 1 2 2 2 2 2 2 5 5 5 3 3 3 5 5 5 5 5 5]
0.3333333333333333


In [116]:
codestrain = authors[intrain]
codestest = authors[notintrain]
xtrain = collate_into_cnbow(list(train_df['text']))
xtest = collate_into_cnbow(list(test_df['text']))
clf = GaussianNB()
clf.fit(xtrain, codestrain)
clf.score(xtest, codestest)

0.3333333333333333